# Анализ пользовательской активности и удержания для оптимизации продукта

**Описание проекта:**
Проект направлен на анализ ключевых метрик пользовательской активности и удержания для оценки эффективности продукта и выявления точек роста. В рамках проекта будут рассчитаны следующие показатели:

* retention
* rolling retention
* lifetime
* churn rate
* mau
* wau
* dau

Файлами для работы являются `registrations.csv` и `entries.csv`. В них хранятся данные о регистрациях пользователей и входа на платформу соответственно.

# Импорт библиотек

In [ ]:
import pandas as pd

# Загрузка данных

In [ ]:
!wget https://gist.github.com/Vs8th/739269a03f2f4a7396d04d6739da3771/raw/registrations.csv

!wget https://gist.github.com/Vs8th/aacb80595d1d6aaa2e31eb735f8bc644/raw/entries.csv

!wget https://gist.github.com/Vs8th/0e827e9a608117345dd6585ab81e8c86/raw/metrics.txt

--2025-03-24 12:01:02--  https://gist.github.com/Vs8th/739269a03f2f4a7396d04d6739da3771/raw/registrations.csv
Resolving gist.github.com (gist.github.com)... 20.27.177.113
Connecting to gist.github.com (gist.github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://gist.githubusercontent.com/Vs8th/739269a03f2f4a7396d04d6739da3771/raw/registrations.csv [following]
--2025-03-24 12:01:03--  https://gist.githubusercontent.com/Vs8th/739269a03f2f4a7396d04d6739da3771/raw/registrations.csv
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14918 (15K) [text/plain]
Saving to: ‘registrations.csv.2’

registrations.csv.2 100%[===================>]  14.57K  --.-KB/s    in 0.004s  

2025-03-24 12:01:03 (3.

In [ ]:
registrations = pd.read_csv('registrations.csv', sep = ';')
entries = pd.read_csv('entries.csv', sep = ';')

In [ ]:
registrations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   user_id            1000 non-null   int64 
 1   registration_date  1000 non-null   object
dtypes: int64(1), object(1)
memory usage: 15.8+ KB


In [ ]:
entries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20734 entries, 0 to 20733
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   user_id     20734 non-null  int64 
 1   entry_date  20734 non-null  object
dtypes: int64(1), object(1)
memory usage: 324.1+ KB


# Расчет метрик

### **Retention 15 дня**

Retention 15 дня (в процентах) для пользователей, зарегистрированных в январе

**Retention (Удержание пользователей)** — процент пользователей, которые вернулись на платформу через определенное время после регистрации от всех зарегистрированных пользователей. Например, Retention 7-го дня показывает, сколько пользователей из зарегистрированных вернулись к нам через неделю после регистрации.

In [ ]:
# Преобразование дат в формат datetime
registrations['registration_date'] = pd.to_datetime(registrations['registration_date'])
entries['entry_date'] = pd.to_datetime(entries['entry_date'])

In [ ]:
# Фильтрация пользователей, зарегистрированных в январе
january_registrations = registrations[registrations['registration_date'].dt.month == 1]

In [ ]:
# Объединение данных о регистрациях и входах
january_merged_data = pd.merge(january_registrations, entries, on='user_id', how='left')

In [ ]:
# Расчет разницы в днях между входом и регистрацией
january_merged_data['days_since_registration'] = (january_merged_data['entry_date'] - january_merged_data['registration_date']).dt.days

In [ ]:
# Проверка, был ли вход на 15 день
january_merged_data['retention_15_day'] = january_merged_data['days_since_registration'] == 15

In [ ]:
# Группировка по пользователям и проверка, был ли у них вход на 15 день
retention_15_day = january_merged_data.groupby('user_id')['retention_15_day'].any()

In [ ]:
# Расчет Retention 15 дня в процентах
retention_15_day = round(retention_15_day.mean() * 100, 5)

In [ ]:
print(f"Retention 15 дня для пользователей, зарегистрированных в январе: {retention_15_day:.2f}%")

Retention 15 дня для пользователей, зарегистрированных в январе: 54.65%


### Rolling-retention

Rolling-retention 30 дня (в процентах) для пользователей из той же когорты

**Rolling Retention (Скользящее удержание)** — похоже на обычное удержание, но учитывает пользователей, которые вернулись на платформу не только в конкретный день, но и позже. Например, Rolling Retention на 7-й день показывает, сколько пользователей вернулись на платформу через 7 дней или позже.

In [ ]:
# Проверка, был ли вход на 30 день или позже
january_merged_data['rolling_retention_30_day'] = january_merged_data['days_since_registration'] >= 30

# Группировка по пользователям и проверка, был ли у них вход на 30 день или позже
rolling_retention_30_day = january_merged_data.groupby('user_id')['rolling_retention_30_day'].any()

# Расчет Retention 30 дня в процентах
rolling_retention_30_day = round(rolling_retention_30_day.mean() * 100, 5)

print(f"Rolling Retention 30 дня для пользователей, зарегистрированных в январе: {rolling_retention_30_day:.2f}%")

Rolling Retention 30 дня для пользователей, зарегистрированных в январе: 29.07%


### Lifetime

Lifetime по всем пользователям, посчитанный как интеграл от n-day retention.

**Lifetime (Средний срок жизни пользователя)** — среднее время, которое пользователь остается активным на платформе, прежде чем перестать заходить. Можем рассчитать как сумму Retention каждого дня. Например, для каждого дня считаем отношение: количество пользователей, вернувшихся в день N, к количеству зарегистрированных пользователей. Затем суммируем эти значения.

In [ ]:
# Объединение данных о регистрациях и входах
df = pd.merge(registrations, entries, on='user_id', how='left')

# Общее число зарегистрированных пользователей (знаменатель для Retention)
total_users = df['user_id'].nunique()

# Расчет разницы в днях между входом и регистрацией
df['days_since_registration'] = (df['entry_date'] - df['registration_date']).dt.days

# Группировка по дням и подсчёт уникальных пользователей в каждый день
daily_active_users = df.groupby('days_since_registration')['user_id'].nunique()

# Расчёт Retention для каждого дня и суммирование (Lifetime)
retention_series = daily_active_users / total_users
lifetime = round(retention_series.sum(), 5)

print(f"Lifetime: {lifetime:.2f}")

Lifetime: 14.80


### Churn rate 29 дня

Churn rate 29 дня (в долях), посчитанный по всем пользователям.

**Churn Rate (Коэффициент оттока)** — процент пользователей, которые перестали пользоваться платформой за определенный период. Например, если из 100 пользователей 10 больше к нам не вернулись спустя месяц, то Churn Rate за этот месяц составит 10%.

In [ ]:
# Проверка, был ли вход на 29 день
df['churn_29'] = df['days_since_registration'] >= 29

# Группировка по пользователям
churn_29 = 1 - df.groupby('user_id')['churn_29'].any().mean()

print(f"Churn Rate 29 дня (в долях), посчитанный по всем пользователям: {churn_29*100:.2f}%")

Churn Rate 29 дня (в долях), посчитанный по всем пользователям: 50.90%


### Mau, Wau, Dau

Mau, Wau, Dau за последний месяц/неделю/день записей

**Примечание:** `mau` рассчитываем для декабря (2021 года), для `wau` берем последнюю неделю - с 25 по 31 декабря, и для `dau` соответственно последний день - 31 декабря.

MAU (Monthly Active Users, Месячная аудитория) — количество уникальных пользователей, которые зашли на платформу за последний месяц.

WAU (Weekly Active Users, Недельная аудитория) — количество уникальных пользователей, которые зашли на платформу за последнюю неделю.

DAU (Daily Active Users, Ежедневная аудитория) — количество уникальных пользователей, которые зашли на платформу за последний день.

In [ ]:
dec_mau = df[df['entry_date'].dt.month == 12]['user_id'].nunique()
print(f'MAU:{dec_mau}')

MAU:133


In [ ]:
dec_wau = df[df['entry_date'].between('2021-12-25', '2021-12-31')]['user_id'].nunique()
print(f'WAU:{dec_wau}')

WAU:84


In [ ]:
dec_dau = df[df['entry_date'] == '2021-12-31']['user_id'].nunique()
print(f'DAU:{dec_dau}')

DAU:47


### Mau, Wau, Dau усредненные


**Примечание:** результаты округлите до 5 знаков после запятой. Сохранить результат в переменные `avg_mau`, `avg_wau`, `avg_dau` соответственно

Усредненные MAU/WAU/DAU - например, для MAU: рассчитываем MAU каждого месяца наблюдений, а затем находим среднее значение по всем месяцам. Аналогично для WAU и DAU.

In [ ]:
avg_mau = df.groupby(df['entry_date'].dt.to_period('M'))['user_id'].nunique().mean()
avg_mau = round(avg_mau, 5)
print(f'Усредненный MAU:{avg_mau:.2f}')

Усредненный MAU:102.58


In [ ]:
avg_wau = df.groupby(df['entry_date'].dt.to_period('W'))['user_id'].nunique().mean()
avg_wau = round(avg_wau, 5)
print(f'Усредненный WAU:{avg_wau:.2f}')

Усредненный WAU:89.87


In [ ]:
avg_dau = df.groupby(df['entry_date'].dt.to_period('D'))['user_id'].nunique().mean()
avg_dau = round(avg_dau, 5)
print(f'Усредненный DAU:{avg_dau:.2f}')

Усредненный DAU:40.56


# Итоговая таблица

In [ ]:
metrics = {'retention_15_day': retention_15_day,
           'rolling_retention_30_day': rolling_retention_30_day,
           'lifetime': lifetime,
           'churn_29': churn_29,
           'dec_mau': dec_mau,
           'avg_mau': avg_mau,
           'dec_wau': dec_wau,
           'avg_wau': avg_wau,
           'dec_dau': dec_dau,
           'avg_dau': avg_dau}

In [ ]:
metrics = pd.DataFrame.from_dict(metrics, orient='index', columns=['value'])
metrics

,value
retention_15_day,54.65116
rolling_retention_30_day,29.06977
lifetime,14.80400
churn_29,0.50900
dec_mau,133.00000
avg_mau,102.58333
dec_wau,84.00000
avg_wau,89.86792
dec_dau,47.00000
avg_dau,40.55890


**Вывод по метрикам:**

Показатели демонстрируют умеренную вовлеченность пользователей:

**Удержание:**

54.65% пользователей возвращаются через 15 дней, но к 30 дням показатель падает до 29.07%, что указывает на потерю интереса в долгосрочной перспективе.

Высокий churn_29 (50.9%) подтверждает, что половина пользователей уходит почти через месяц.

**Активность (DAU/WAU/MAU):**

- Средний DAU (40.56) значительно ниже декабрьского (47), возможен сезонный рост.

- WAU (89.87) близок к декабрьскому (84), но MAU (102.58 vs. 133 в декабре) показывает снижение месячной аудитории.

- Lifetime (14.8 дней):
Короткий цикл жизни пользователей требует улучшения retention-стратегий, особенно после 15 дней.

**Рекомендации:** Улучшать долгосрочное удержание (например, через триггерные уведомления, контент-маркетинг), анализировать причины падения MAU и тестировать методы reactivation.